# 🔀 Symmetry Decomposition: V₊ ⊕ V₋

**The Foundation:** Every involution $\sigma: V \to V$ with $\sigma^2 = I$ splits the space into eigenspaces.

---

## 📐 Mathematical Setup

For an involution $\sigma$ (self-inverse: $\sigma^2 = I$), **any** vector $x \in \mathbb{R}^n$ decomposes uniquely as:

$$x = x_+ + x_-$$

where:
- $x_+ = \frac{1}{2}(x + \sigma(x)) \in V_+$ (symmetric component, eigenvalue $+1$)
- $x_- = \frac{1}{2}(x - \sigma(x)) \in V_-$ (antisymmetric component, eigenvalue $-1$)

**Properties:**
1. $\sigma(x_+) = +x_+$ (symmetric: unchanged)
2. $\sigma(x_-) = -x_-$ (antisymmetric: sign flip)
3. $\langle x_+, x_- \rangle = 0$ (orthogonal)
4. $\|x\|^2 = \|x_+\|^2 + \|x_-\|^2$ (Pythagorean theorem)

---

## 📊 Coherence Parameter

The **coherence** $\alpha$ measures the fraction of energy in the symmetric subspace:

$$\alpha = \frac{\|x_+\|^2}{\|x\|^2} = \frac{\|x_+\|^2}{\|x_+\|^2 + \|x_-\|^2} \in [0, 1]$$

**Interpretation:**
- $\alpha = 1$: Fully symmetric ($x = x_+$, $x_- = 0$)
- $\alpha = 0$: Fully antisymmetric ($x = x_-$, $x_+ = 0$)
- $\alpha = 0.5$: Balanced ($\|x_+\| = \|x_-\|$)

---

## 🎯 Learning Objectives

1. **Visualize** the decomposition for different involutions
2. **Understand** how $\alpha$ relates to symmetry strength
3. **Explore** antipodal, reverse, and reflection involutions
4. **Verify** orthogonality and reconstruction
5. **Interactive** demos with real data

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider, Dropdown, IntSlider
import ipywidgets as widgets

import sys
sys.path.insert(0, '..')

from quotient_probes import SymmetryProbe
from quotient_probes.core.involutions import antipodal, reverse, reflection
from quotient_probes.core.decomposition import (
    decompose, decompose_symmetric, decompose_antisymmetric,
    compute_coherence, verify_decomposition
)

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("✅ Setup complete!")

---

## 🔍 Example 1: Antipodal Involution

**Antipodal:** $\sigma(x) = -x$

This is the simplest involution. Let's see how it decomposes vectors.

In [ ]:
# Example vector
x = np.array([3.0, 1.0, 2.0])

# Decompose
x_plus, x_minus = decompose(x, antipodal)

print("Antipodal Decomposition: σ(x) = -x")
print("="*50)
print(f"Original:            x = {x}")
print(f"Symmetric component: x₊ = {x_plus}")
print(f"Antisymmetric:       x₋ = {x_minus}")
print()

# Verify properties
print("Verification:")
print(f"  σ(x₊) = {antipodal(x_plus)} (should equal +x₊)")
print(f"  σ(x₋) = {antipodal(x_minus)} (should equal -x₋)")
print(f"  x₊ + x₋ = {x_plus + x_minus} (should equal x)")
print(f"  ⟨x₊, x₋⟩ = {np.dot(x_plus, x_minus):.6f} (should be 0)")
print()

# Compute coherence
alpha = compute_coherence(x, antipodal)
print(f"Coherence: α = {alpha:.6f}")
print(f"Energy split: {alpha*100:.2f}% in V₊, {(1-alpha)*100:.2f}% in V₋")

**Key Observation:** For antipodal involution, generic vectors are **purely antisymmetric** ($x_+ = 0$, $\alpha = 0$).
Only the zero vector is symmetric under $x \mapsto -x$.

---

## 🔄 Example 2: Reverse Involution (Time Reversal)

**Reverse:** $\sigma(x) = x[::-1]$ (flip order)

This is perfect for time series! Palindromes are symmetric.

In [ ]:
# Palindrome example
x_palindrome = np.array([1, 2, 3, 2, 1], dtype=float)

x_plus_p, x_minus_p = decompose(x_palindrome, reverse)
alpha_p = compute_coherence(x_palindrome, reverse)

print("Palindrome (Fully Symmetric):")
print("="*50)
print(f"x = {x_palindrome}")
print(f"x₊ = {x_plus_p}")
print(f"x₋ = {x_minus_p}")
print(f"α = {alpha_p:.6f} (fully symmetric!)\n")

# Anti-palindrome example
x_anti = np.array([1, 2, 0, -2, -1], dtype=float)

x_plus_a, x_minus_a = decompose(x_anti, reverse)
alpha_a = compute_coherence(x_anti, reverse)

print("Anti-palindrome (Fully Antisymmetric):")
print("="*50)
print(f"x = {x_anti}")
print(f"x₊ = {x_plus_a}")
print(f"x₋ = {x_minus_a}")
print(f"α = {alpha_a:.6f} (fully antisymmetric!)\n")

# Generic example
x_generic = np.array([1, 3, 2, 4, 5], dtype=float)

x_plus_g, x_minus_g = decompose(x_generic, reverse)
alpha_g = compute_coherence(x_generic, reverse)

print("Generic vector (Mixed):")
print("="*50)
print(f"x = {x_generic}")
print(f"x₊ = {x_plus_g}")
print(f"x₋ = {x_minus_g}")
print(f"α = {alpha_g:.6f} (mixed!)")

---

## 📈 Visualizing Decomposition

Let's visualize how a time series decomposes under reverse involution.

In [ ]:
def visualize_decomposition(data, involution_name='reverse'):
    """Visualize symmetry decomposition."""
    if involution_name == 'reverse':
        sigma = reverse
    elif involution_name == 'antipodal':
        sigma = antipodal
    else:
        raise ValueError(f"Unknown involution: {involution_name}")
    
    # Decompose
    x_plus, x_minus = decompose(data, sigma)
    alpha = compute_coherence(data, sigma)
    
    # Plot
    fig, axes = plt.subplots(4, 1, figsize=(12, 10))
    
    # Original signal
    ax = axes[0]
    ax.plot(data, 'k-', linewidth=2, marker='o', markersize=6)
    ax.set_title(f'Original Signal x (Involution: {involution_name})', fontsize=13, fontweight='bold')
    ax.set_ylabel('Value', fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)
    
    # Symmetric component
    ax = axes[1]
    ax.plot(x_plus, 'b-', linewidth=2, marker='s', markersize=6, label='x₊ (symmetric)')
    ax.set_title(f'Symmetric Component x₊ ∈ V₊ (||x₊||² = {np.linalg.norm(x_plus)**2:.2f})', 
                fontsize=13, fontweight='bold', color='blue')
    ax.set_ylabel('Value', fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)
    ax.legend(fontsize=10)
    
    # Antisymmetric component
    ax = axes[2]
    ax.plot(x_minus, 'r-', linewidth=2, marker='^', markersize=6, label='x₋ (antisymmetric)')
    ax.set_title(f'Antisymmetric Component x₋ ∈ V₋ (||x₋||² = {np.linalg.norm(x_minus)**2:.2f})', 
                fontsize=13, fontweight='bold', color='red')
    ax.set_ylabel('Value', fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)
    ax.legend(fontsize=10)
    
    # Reconstruction
    ax = axes[3]
    ax.plot(data, 'k-', linewidth=2, marker='o', markersize=6, label='Original x', alpha=0.7)
    ax.plot(x_plus + x_minus, 'g--', linewidth=2, marker='D', markersize=5, 
           label='Reconstruction x₊ + x₋', alpha=0.7)
    ax.set_title(f'Reconstruction Verification (α = {alpha:.4f})', fontsize=13, fontweight='bold')
    ax.set_xlabel('Index', fontsize=11)
    ax.set_ylabel('Value', fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)
    ax.legend(fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n{'='*60}")
    print(f"DECOMPOSITION STATISTICS")
    print(f"{'='*60}")
    print(f"Coherence α:          {alpha:.6f}")
    print(f"Energy in V₊:         {alpha*100:.2f}%")
    print(f"Energy in V₋:         {(1-alpha)*100:.2f}%")
    print(f"||x||²:               {np.linalg.norm(data)**2:.4f}")
    print(f"||x₊||²:              {np.linalg.norm(x_plus)**2:.4f}")
    print(f"||x₋||²:              {np.linalg.norm(x_minus)**2:.4f}")
    print(f"||x₊||² + ||x₋||²:    {np.linalg.norm(x_plus)**2 + np.linalg.norm(x_minus)**2:.4f}")
    print(f"⟨x₊, x₋⟩:             {np.dot(x_plus, x_minus):.10f}")
    print(f"Reconstruction error: {np.linalg.norm(data - (x_plus + x_minus)):.2e}")
    print(f"{'='*60}\n")

# Test with sinusoidal data
t = np.linspace(0, 4*np.pi, 50)
signal = np.sin(t) + 0.3 * np.sin(3*t)

visualize_decomposition(signal, 'reverse')

---

## 🎮 Interactive Explorer

**Explore different signals and involutions!**

In [ ]:
def interactive_decomposition(signal_type, n_points, symmetry_strength, involution_name):
    """Interactive decomposition explorer."""
    
    # Generate signal
    t = np.linspace(0, 4*np.pi, n_points)
    
    if signal_type == 'Sine wave':
        base_signal = np.sin(t)
    elif signal_type == 'Cosine wave':
        base_signal = np.cos(t)
    elif signal_type == 'Sawtooth':
        base_signal = 2 * (t / (4*np.pi) - np.floor(t / (4*np.pi) + 0.5))
    elif signal_type == 'Random':
        np.random.seed(42)
        base_signal = np.random.randn(n_points)
    elif signal_type == 'Pulse':
        base_signal = np.where((t > np.pi) & (t < 3*np.pi), 1.0, 0.0)
    else:
        base_signal = np.sin(t)
    
    # Add symmetric component to control α
    if involution_name == 'reverse':
        # Make signal more symmetric by blending with reversed version
        signal = (1 - symmetry_strength) * base_signal + symmetry_strength * base_signal[::-1]
    else:
        signal = base_signal
    
    # Visualize
    visualize_decomposition(signal, involution_name)

# Interactive widget
interact(
    interactive_decomposition,
    signal_type=Dropdown(
        options=['Sine wave', 'Cosine wave', 'Sawtooth', 'Random', 'Pulse'],
        value='Sine wave',
        description='Signal:'
    ),
    n_points=IntSlider(min=10, max=100, step=5, value=50, description='Points:'),
    symmetry_strength=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.3, description='Symmetry:'),
    involution_name=Dropdown(
        options=['reverse', 'antipodal'],
        value='reverse',
        description='Involution:'
    )
);

---

## 📊 Energy Distribution Visualization

Let's visualize how coherence $\alpha$ relates to energy distribution.

In [ ]:
def plot_energy_distribution(n=50, num_samples=100):
    """Plot energy distribution for random vectors."""
    
    np.random.seed(42)
    
    alphas = []
    norm_ratios = []
    
    for _ in range(num_samples):
        # Generate random signal
        x = np.random.randn(n)
        
        # Decompose under reverse involution
        x_plus, x_minus = decompose(x, reverse)
        alpha = compute_coherence(x, reverse)
        
        alphas.append(alpha)
        norm_ratios.append(np.linalg.norm(x_plus) / (np.linalg.norm(x) + 1e-10))
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Histogram of coherence
    ax = axes[0]
    ax.hist(alphas, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    ax.axvline(np.mean(alphas), color='red', linestyle='--', linewidth=2, label=f'Mean = {np.mean(alphas):.3f}')
    ax.axvline(0.5, color='green', linestyle='--', linewidth=2, label='Balanced (α=0.5)')
    ax.set_xlabel('Coherence α', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.set_title(f'Distribution of Coherence (n={n}, {num_samples} samples)', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Scatter: alpha vs norm ratio
    ax = axes[1]
    ax.scatter(alphas, norm_ratios, alpha=0.5, s=30)
    ax.plot([0, 1], [0, 1], 'r--', linewidth=2, label='α = ||x₊||/||x||')
    ax.set_xlabel('Coherence α = ||x₊||²/||x||²', fontsize=12)
    ax.set_ylabel('Norm ratio ||x₊||/||x||', fontsize=12)
    ax.set_title('Coherence vs Norm Ratio', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    # Energy split pie chart (average)
    ax = axes[2]
    avg_alpha = np.mean(alphas)
    ax.pie([avg_alpha, 1-avg_alpha], labels=['V₊ (symmetric)', 'V₋ (antisymmetric)'],
          autopct='%1.1f%%', colors=['blue', 'red'], startangle=90)
    ax.set_title(f'Average Energy Split\n(reverse involution on random data)', fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nStatistics for {num_samples} random vectors (n={n}):")
    print(f"  Mean coherence:   {np.mean(alphas):.4f}")
    print(f"  Std coherence:    {np.std(alphas):.4f}")
    print(f"  Min coherence:    {np.min(alphas):.4f}")
    print(f"  Max coherence:    {np.max(alphas):.4f}")
    print(f"\n💡 Random data typically has α ≈ 0.5 (balanced energy split)")

plot_energy_distribution(n=50, num_samples=100)

---

## 🧪 Using the SymmetryProbe API

The library provides a high-level API for symmetry analysis.

In [ ]:
# Generate test signal
t = np.linspace(0, 4*np.pi, 64)
signal = np.sin(t) + 0.5 * np.sin(2*t)

# Create probe
probe = SymmetryProbe(signal, involution='reverse')

# Analyze
alpha, bit_savings, should_exploit = probe.analyze()

# Get components
x_plus, x_minus = probe.decompose()

# Print summary
print(probe.summary())

# Verify
is_valid = probe.verify()
print(f"\n✅ Decomposition is valid: {is_valid}")

---

## 🎓 Key Takeaways

1. **Every involution** $\sigma$ splits the space: $V = V_+ \oplus V_-$

2. **Decomposition is unique**: $x = x_+ + x_-$ with:
   - $x_+ = \frac{1}{2}(x + \sigma(x))$
   - $x_- = \frac{1}{2}(x - \sigma(x))$

3. **Coherence** $\alpha$ measures symmetry strength:
   - $\alpha \to 1$: Data is symmetric
   - $\alpha \to 0$: Data is antisymmetric
   - $\alpha \approx 0.5$: Random data (balanced)

4. **Components are orthogonal**: $\langle x_+, x_- \rangle = 0$

5. **Energy is conserved**: $\|x\|^2 = \|x_+\|^2 + \|x_-\|^2$

6. **Different involutions** reveal different symmetries:
   - Antipodal: Sign symmetry
   - Reverse: Time-reversal symmetry
   - Reflection: Spatial symmetry

---

## 🚀 Next Steps

- **Notebook 02**: Learn when to exploit this decomposition (MDL decision rule)
- **Notebook 03**: Understand orientation cost (Bernoulli vs Markov)
- **Notebook 04**: See real-world applications

---

## 📚 Mathematical References

- **Spectral Theorem**: Any self-adjoint operator decomposes into eigenspaces
- **Representation Theory**: Quotient spaces $V/\sigma$ and lift maps
- **Functional Analysis**: Projection operators $P_\pm = \frac{1}{2}(I \pm \sigma)$